<a href="https://colab.research.google.com/github/geansm2/Data_Analytics/blob/main/Cappta_Gean_Machado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coleta os dados do IDHM por cidade

In [ ]:
import pandas as pd
from pandas import DataFrame

url = 'https://www.br.undp.org/content/brazil/pt/home/idh0/rankings/idhm-municipios-2010.html'

tabela = pd.read_html(url)
idh = pd.DataFrame(tabela[0])

#seprar o marcador de unidade federativa informada no nome do munícipio
idh[['Município', 'UF']] = idh['Município'].str.split('(', 1, expand=True)
idh['UF'] = [x.split(')')[0] for x in idh['UF']]
#remove espaço na ultima posição da string municipio
idh['Município'] = idh['Município'].map(lambda x: str(x)[:-1])

#filtra as variáveis de interesse
del idh['Ranking IDHM 2010']
del idh['IDHM Educação 2010']
del idh['IDHM Longevidade 2010']
del idh['IDHM Renda 2010']

#corrige o idh para decimal
idh['IDHM 2010'] = '0.'+idh['IDHM 2010'].astype(str)

idh

,Município,IDHM 2010,UF
0,São Caetano do Sul,0.862,SP
1,Águas de São Pedro,0.854,SP
2,Florianópolis,0.847,SC
3,Balneário Camboriú,0.845,SC
4,Vitória,0.845,ES
...,...,...,...
5560,Uiramutã,0.453,RR
5561,Marajá do Sena,0.452,MA
5562,Atalaia do Norte,0.450,AM
5563,Fernando Falcão,0.443,MA


Se a saída acima tiver 5565 linhas está tudo certo, pode prosseguir

# Coleta os dados para usuários pós pago por estado
filtra, ajusta e corrige incossistências 

In [ ]:

url = 'https://www.teleco.com.br/pospago_uf.asp'

pos = pd.read_html(url)
#idenfificado as posições dos objetos anteriormente 
df1 = pos[109]
df2 = pos[110]
df3 = pos[111]

#mescla os objetos numa unica tabela
frames = [df1, df2, df3]
pos_pago = pd.concat(frames)


#filtra apenas os dados de interresse
cols = ['UF', 'Total']
pos_pago_uf = pos_pago[cols]

#a base de dados tem um padrão direfente para as observações de DF e GO, aqui se faz necessário um ajuste
pos_pago_uf.iat[19,0] = 'DF'
pos_pago_uf.iat[20,0] = 'GO'
pos_pago_uf


,UF,Total
0,RJ,11.365.966
1,ES,2.315.813
2,MG,11.664.480
3,BA,4.903.670
4,SE,782.290
5,PE,3.499.936
6,AL,934.277
7,PB,1.453.657
8,RN,1.169.578
9,CE,3.433.157


Junta-se as tabela de IDHM com a tabela de celulares pós pago

In [ ]:
#junta a tabela pos_pago_uf e idh pela id UF
df_merge = pd.merge(pos_pago_uf, idh, how = 'outer', on = 'UF')

df_merge.columns = df_merge.columns.str.rstrip()
df_merge.columns = df_merge.columns.str.lstrip()
df_merge

,UF,Total,Município,IDHM 2010
0,RJ,11.365.966,Niterói,0.837
1,RJ,11.365.966,Rio de Janeiro,0.799
2,RJ,11.365.966,Rio das Ostras,0.773
3,RJ,11.365.966,Volta Redonda,0.771
4,RJ,11.365.966,Resende,0.768
...,...,...,...,...
5560,SP,46.895.391,Redenção da Serra,0.657
5561,SP,46.895.391,Natividade da Serra,0.655
5562,SP,46.895.391,Nova Campina,0.651
5563,SP,46.895.391,Barra do Turvo,0.641


# Coleta os dados demográficos

In [ ]:
# import pandas as pd
# from pandas import DataFrame

url = 'https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil_por_popula%C3%A7%C3%A3o_(2020)'

tabela = pd.read_html(url)
df_pop_municipio = pd.DataFrame(tabela[0])

#filtra varáveis de interesse
del df_pop_municipio['Posição']
del df_pop_municipio['Código IBGE']
del df_pop_municipio['Unidade federativa']

#retira os espaços das string municipio
df_pop_municipio.columns = df_pop_municipio.columns.str.rstrip()

# existe uma sensibilidade ao fazer join com a coluna Municipio então precisa ajustar o nome das cidade
df_merge.columns = [c.lower() for c in df_merge.columns]
df_pop_municipio.columns = [c.lower() for c in df_pop_municipio.columns]

df_merge['município'] = df_merge['município'].str.lower()
df_pop_municipio['município']=df_pop_municipio['município'].str.lower()

# #junta as tabela da população municipal com a tabela mesclada anterior pela id municipio
table_final = pd.merge(df_pop_municipio,df_merge, how = 'outer', on = 'município')

#Se população tem NaN excluia toda a linha
table_final.dropna(subset = ['população'], inplace=True)
#renomeia
table_final = table_final.rename(columns={'população': 'populacaomunicipal', 'total': 'N_pos_pago', 'idhm 2010': 'IDHM'})
table_final

,município,populacaomunicipal,uf,N_pos_pago,IDHM
0,são paulo,12 325 232,SP,46.895.391,0.805
1,rio de janeiro,6 747 815,RJ,11.365.966,0.799
2,brasília,3 055 149,DF,2.230.542,0.824
3,salvador,2 886 698,BA,4.903.670,0.759
4,fortaleza,2 686 612,CE,3.433.157,0.754
...,...,...,...,...,...
6214,união da serra,1 118,RS,7.763.486,0.733
6215,engenho velho,982,RS,7.763.486,0.717
6216,araguainha,946,MT,1.870.578,0.701
6217,borá,838,SP,46.895.391,0.746


# Modelagem dos dados e inferência

In [ ]:
#modelagem e ajuste das variaveis numéricas
import numpy as np

#remove non-breaking space e corrige formato numérico
table_final.loc[:,'populacaomunicipal']= table_final['populacaomunicipal'].apply(lambda x: str(x).replace('\u00a0', '')).astype(float)
table_final.loc[:,'N_pos_pago']= table_final['N_pos_pago'].apply(lambda x: str(x).replace('.', '')).astype(float)
table_final.loc[:,'IDHM']= table_final['IDHM'].apply(lambda x: str(x).replace(',', '.')).astype(float)

#cria a coluna de Coeficiente de ajuste populacional municipal
conditions = [
    (table_final['populacaomunicipal'] < 5001),
    (table_final['populacaomunicipal'] < 20001),
    (table_final['populacaomunicipal'] < 100001),
    (table_final['populacaomunicipal'] < 500001),
    (table_final['populacaomunicipal'] > 500001)]
choices = [5, 10, 15, 20, 25]

table_final['Cp'] = np.select(conditions, choices)

#criar coluna de total populacional estadual
temp = table_final.groupby(['uf']).agg({'populacaomunicipal': 'sum'})
temp['UF'] = temp.index

#retira o nome do index
temp.index.name  = None

# renomeai a coluna para melhor entender os dados
temp = temp.rename(columns={'populacaomunicipal': 'populacaoestadual', 'UF':'uf'})

#junta as tabela da população estadual com a tabela final anterior
table_final_2 = pd.merge(table_final,temp, how = 'outer', on = 'uf')

#Se população tem NaN excluia toda a linha
table_final_2.dropna(subset = ['uf'], inplace=True)

#cria a Teledensidade estadual de celulares pós pagos
table_final_2['Td'] = (table_final_2.N_pos_pago / table_final_2.populacaoestadual) 

#cria Probabilidade de conversao e calcula numero de clientes convertidos
table_final_2['PC'] = ((table_final_2.IDHM ** table_final_2.Cp) * ((table_final_2.Td/1.5)*100) )
table_final_2['N_cliente_convertido'] = (table_final_2.PC * table_final_2.populacaomunicipal) / 100

#exibe a tabela e ordena por numero de cliente convertido
table_final_2.sort_values(by='N_cliente_convertido', ascending=False)

,município,populacaomunicipal,uf,N_pos_pago,IDHM,Cp,populacaoestadual,Td,PC,N_cliente_convertido
0,são paulo,12325232.0,SP,46895391.0,0.805,25,46590748.0,1.006539,2.962349e-01,36511.639206
789,brasília,3055149.0,DF,2230542.0,0.824,25,3055149.0,0.730093,3.850056e-01,11762.494641
688,rio de janeiro,6747815.0,RJ,11365966.0,0.799,25,17791920.0,0.638827,1.559415e-01,10522.645184
13,santos,433656.0,SP,46895391.0,0.840,20,46590748.0,1.006539,2.052697e+00,8901.645605
15,jundiaí,423006.0,SP,46895391.0,0.822,20,46590748.0,1.006539,1.330992e+00,5630.177638
...,...,...,...,...,...,...,...,...,...,...
3876,chaves,23948.0,PA,2291158.0,0.453,15,8805722.0,0.260190,1.204126e-04,0.028836
2390,atalaia do norte,20398.0,AM,1346024.0,0.450,15,4255601.0,0.316295,1.324916e-04,0.027026
3788,breves,103497.0,PA,2291158.0,0.503,20,8805722.0,0.260190,1.864483e-05,0.019297
3865,melgaço,27890.0,PA,2291158.0,0.418,15,8805722.0,0.260190,3.604483e-05,0.010053


# Classificação das cidades desbancarizadas
Aqui salvamos as planilhas disponibilizada pelo BCB no google driver, é apartir de que faremos a leitura dos dados
# agencias

In [ ]:
#instala o PyDrive wrapper e importa as bibliotecas.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#cria atuenticação e credencia a aplicação para acesso a planilha 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#indicação da planilha
file_id = '1DIHW8haHsDI6H9iyqpFNiUBzt6Q6jYmz'
downloaded = drive.CreateFile({'id': file_id})



In [ ]:
#donwload do arquivo para o disco local
downloaded.GetContentFile('agencias.xlsx')

In [ ]:
#confere o arquivo
!ls -lha agencias.xlsx

-rw-r--r-- 1 root root 2.3M Jul 30 13:11 agencias.xlsx


In [ ]:
#agora, podemos fazer a leitura do arquivo do excel com o pandas
#garante que o suplemento para leitura de arquivo do excel esteja disponivel
!pip install -q xlrd
import pandas as pd

#leitura e subset
df = pd.read_excel('agencias.xlsx')
df = df.rename(columns={'MUNICíPIO                                                   ': 'Município'})
sub_df = df[['Município', 'UF']]

#contagem de agencias por municípios
agencias=sub_df.groupby(['Município']).size()

# converter nome do municipio para letra minuscula
agencia_df = pd.DataFrame(agencias)
agencia_df['Município'] = agencia_df.index
#retira o nome do index
agencia_df.index.name  = None

#renomeia as colunas e garente que o nome das cidades estaram em minusculo
agencia_df.columns = ['n_agencias','Município']
agencia_df['Município']=agencia_df['Município'].str.lower()

#remover os espacos na string municipio
agencia_df['Município'] = agencia_df['Município'].apply(lambda x:x.strip() )

agencia_df

,n_agencias,Município
ABADIANIA,2,abadiania
ABAETE,3,abaete
ABAETETUBA,7,abaetetuba
ABARE,1,abare
ABATIA,1,abatia
...,...,...
XAVANTINA,1,xavantina
XAXIM,3,xaxim
XINGUARA,5,xinguara
XIQUE-XIQUE,4,xique-xique


Para continuar fazendo join nas tabelas é preciso ajustar as variáveis da coluna municípo para o mesmo padrão

In [ ]:
#retira os ascentos do municipios listado na tabela 2
from unicodedata import normalize

#função para remover os ascentos no nome das cidade
def remover_acentos(txt):
  return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

#ajustes de strings
table_final_2.município = table_final_2.município.astype(str)
agencia_df.columns = [c.lower() for c in agencia_df.columns]
agencia_df.município = agencia_df.município.astype(str)

#remove os ascentos
table_final_2['município'] = table_final_2['município'].apply(remover_acentos)

#junta a tabela de agencias com a tabela 2, criando a tabela 3 
table_final_3 = pd.merge(table_final_2,agencia_df, how = 'outer', on = 'município')
table_final_3

,município,populacaomunicipal,uf,N_pos_pago,IDHM,Cp,populacaoestadual,Td,PC,N_cliente_convertido,n_agencias
0,sao paulo,12325232.0,SP,46895391.0,0.805,25.0,46590748.0,1.006539,0.296235,36511.639206,2170.0
1,guarulhos,1392121.0,SP,46895391.0,0.763,25.0,46590748.0,1.006539,0.077598,1080.263377,101.0
2,campinas,1213792.0,SP,46895391.0,0.805,25.0,46590748.0,1.006539,0.296235,3595.675568,203.0
3,sao bernardo do campo,844483.0,SP,46895391.0,0.805,25.0,46590748.0,1.006539,0.296235,2501.653406,90.0
4,sao jose dos campos,729737.0,SP,46895391.0,0.807,25.0,46590748.0,1.006539,0.315194,2300.085363,75.0
...,...,...,...,...,...,...,...,...,...,...,...
6194,sao lourenco d'oeste,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
6195,sao luis do paraitinga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
6196,sao miguel d'oeste,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
6197,sao tome das letras,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


56 cidades não foram mescladas


# Postos
Uma vez definido os parametros de acesso ao google drive, agora só é necessário indicar o arquivo que precisa ser acessado e usado para coleta

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#indicação da planilha
file_id = '1RY-o5NBI4wnhN4By6sJGBUyTPq7FSyPc'
downloaded = drive.CreateFile({'id': file_id})


In [ ]:

# Download the file to a local disk as 'exported.xlsx'.
downloaded.GetContentFile('postos.xlsx')

In [ ]:

# !ls -lha postos.xlsx

# import pandas as pd
#leitura e subset
df = pd.read_excel('postos.xlsx')
df = df.rename(columns={'MUNICIPIO                                                   ': 'Município'})
sub_df = df[['Município', 'UF']]

#contagem de postos de atendimento
postos=sub_df.groupby(['Município']).size()

## converter nome do municipio para letra minuscula
postos_df = pd.DataFrame(postos)
postos_df['Município'] = postos_df.index
#retira o nome do index
postos_df.index.name  = None

#ajuste de string
postos_df.columns = ['n_postos','Município']
postos_df['Município'] = postos_df['Município'].str.lower()
postos_df['Município'] = postos_df['Município'].apply(lambda x:x.strip() )

postos_df

,n_postos,Município
ABADIA DE GOIAS,2,abadia de goias
ABADIA DOS DOURADOS,2,abadia dos dourados
ABADIANIA,1,abadiania
ABAETE,2,abaete
ABAETETUBA,1,abaetetuba
...,...,...
XAXIM,6,xaxim
XEXEU,1,xexeu
XINGUARA,2,xinguara
ZACARIAS,1,zacarias


In [ ]:
#ajuste e join das tabela postos e tabela 3
postos_df.columns = [c.lower() for c in postos_df.columns]
postos_df.município = postos_df.município.astype(str)

#criação da quarta tabela
table_final_4 = pd.merge(table_final_3,postos_df, how = 'outer', on = 'município')
table_final_4

,município,populacaomunicipal,uf,N_pos_pago,IDHM,Cp,populacaoestadual,Td,PC,N_cliente_convertido,n_agencias,n_postos
0,sao paulo,12325232.0,SP,46895391.0,0.805,25.0,46590748.0,1.006539,0.296235,36511.639206,2170.0,1178.0
1,guarulhos,1392121.0,SP,46895391.0,0.763,25.0,46590748.0,1.006539,0.077598,1080.263377,101.0,113.0
2,campinas,1213792.0,SP,46895391.0,0.805,25.0,46590748.0,1.006539,0.296235,3595.675568,203.0,153.0
3,sao bernardo do campo,844483.0,SP,46895391.0,0.805,25.0,46590748.0,1.006539,0.296235,2501.653406,90.0,95.0
4,sao jose dos campos,729737.0,SP,46895391.0,0.807,25.0,46590748.0,1.006539,0.315194,2300.085363,75.0,101.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6236,serido,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6237,serra caiada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6238,taiuva,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
6239,vespasiano correa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


# Posto de atendimento eletrônico

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#indicação da planilha
file_id = '1eBFDEDxp1LdwEk_HMnQUrYT48UCUVDtf'
downloaded = drive.CreateFile({'id': file_id})


In [ ]:

# Download the file to a local disk as 'exported.xlsx'.
downloaded.GetContentFile('pae.xlsx')


In [ ]:
#leitura e subset
df = pd.read_excel('pae.xlsx')
df = df.rename(columns={'MUNICIPIO                                                   ': 'Município'})
sub_df = df[['Município', 'UF']]

#contagem de p.a.e
pae=sub_df.groupby(['Município']).size()

## converter nome do municipio para letra minuscula
pae_df = pd.DataFrame(pae)
pae_df['Município'] = pae_df.index
#retira o nome do index
pae_df.index.name  = None

pae_df.columns = ['n_pae','Município']

pae_df['Município'] = pae_df['Município'].str.lower()

#remover os espacos na string municipio

pae_df['Município'] = pae_df['Município'].apply(lambda x:x.strip() )
pae_df

,n_pae,Município
ABADIA DE GOIAS,1,abadia de goias
ABADIA DOS DOURADOS,1,abadia dos dourados
ABADIANIA,2,abadiania
ABAETE,6,abaete
ABAETETUBA,15,abaetetuba
...,...,...
XEXEU,1,xexeu
XINGUARA,5,xinguara
XIQUE-XIQUE,3,xique-xique
ZACARIAS,2,zacarias


In [ ]:
#garante que os dados sejam string e estajam e caixa baixa
pae_df.columns = [c.lower() for c in pae_df.columns]
pae_df.município = pae_df.município.astype(str)

#join a tabela de postos de atendimentos eletrônico com a tabela 4
table_final_5 = pd.merge(table_final_4,pae_df, how = 'outer', on = 'município')

#subistitui NA por zero
table_final_5['n_agencias'] = table_final_5['n_agencias'].fillna(0)
table_final_5['n_postos'] = table_final_5['n_postos'].fillna(0)
table_final_5['n_pae'] = table_final_5['n_pae'].fillna(0)

#Cria a coluna com nivel de bancarização
table_final_5['Nb'] = (table_final_5.n_agencias + table_final_5.n_postos + table_final_5.n_pae) / table_final_5.populacaomunicipal

#Se Nb tem NaN excluia toda a linha
table_final_5.dropna(subset = ['Nb'], inplace=True)

#exibe a tabela e ordena por numero de desbacarização
ordenado = table_final_5.sort_values(by='Nb', ascending=True)

#converte o numero de cliente convertido para inteiro
ordenado['N_cliente_convertido'] = ordenado['N_cliente_convertido'].astype(int)

#cria coluna cumulativa para PC
ordenado['cum_sum_pc'] = ordenado['N_cliente_convertido'].cumsum()
ordenado


,município,populacaomunicipal,uf,N_pos_pago,IDHM,Cp,populacaoestadual,Td,PC,N_cliente_convertido,n_agencias,n_postos,n_pae,Nb,cum_sum_pc
6142,oliveira de fatima,1118.0,TO,579944.0,0.675,5.0,2306116.0,0.251481,2.349268,26,0.0,0.0,0.0,0.000000,26
5398,miguel leao,1242.0,PI,939690.0,0.623,5.0,3714860.0,0.252954,1.582673,19,0.0,0.0,0.0,0.000000,45
5397,santo antonio dos milagres,2166.0,PI,939690.0,0.619,5.0,3714860.0,0.252954,1.532513,33,0.0,0.0,0.0,0.000000,78
5396,sao miguel da baixa grande,2454.0,PI,939690.0,0.563,5.0,3714860.0,0.252954,0.953878,23,0.0,0.0,0.0,0.000000,101
5395,olho d'agua do piaui,2468.0,PI,939690.0,0.576,5.0,3714860.0,0.252954,1.069211,26,0.0,0.0,0.0,0.000000,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,santo andre,2509.0,PB,1453657.0,0.600,5.0,9008601.0,0.161363,0.836507,20,79.0,74.0,111.0,0.105221,887125
6,santo andre,2509.0,SP,46895391.0,0.815,5.0,46590748.0,1.006539,24.128330,605,79.0,74.0,111.0,0.105221,887730
3728,belem,4284.0,PA,2291158.0,0.746,5.0,8805722.0,0.260190,4.007679,171,110.0,96.0,409.0,0.143557,887901
3725,belem,4284.0,PB,1453657.0,0.592,5.0,9008601.0,0.161363,0.782207,33,110.0,96.0,409.0,0.143557,887934


# Por critério de baixo nivel de bacarização é possivel atigir o objetivo de converter 1.000 clientes com as seguintes cidades:

In [ ]:
temp = ordenado[ordenado['cum_sum_pc'] < 1336]
temp
# temp.to_csv('lista.csv')

,município,populacaomunicipal,uf,N_pos_pago,IDHM,Cp,populacaoestadual,Td,PC,N_cliente_convertido,n_agencias,n_postos,n_pae,Nb,cum_sum_pc
6142,oliveira de fatima,1118.0,TO,579944.0,0.675,5.0,2306116.0,0.251481,2.349268,26,0.0,0.0,0.0,0.0,26
5398,miguel leao,1242.0,PI,939690.0,0.623,5.0,3714860.0,0.252954,1.582673,19,0.0,0.0,0.0,0.0,45
5397,santo antonio dos milagres,2166.0,PI,939690.0,0.619,5.0,3714860.0,0.252954,1.532513,33,0.0,0.0,0.0,0.0,78
5396,sao miguel da baixa grande,2454.0,PI,939690.0,0.563,5.0,3714860.0,0.252954,0.953878,23,0.0,0.0,0.0,0.0,101
5395,olho d'agua do piaui,2468.0,PI,939690.0,0.576,5.0,3714860.0,0.252954,1.069211,26,0.0,0.0,0.0,0.0,127
5394,pedro laurentino,2544.0,PI,939690.0,0.562,5.0,3714860.0,0.252954,0.945437,24,0.0,0.0,0.0,0.0,151
5393,aroeiras do itaim,2551.0,PI,939690.0,0.519,5.0,3714860.0,0.252954,0.635020,16,0.0,0.0,0.0,0.0,167
4070,riachao do poco,4540.0,PB,1453657.0,0.555,5.0,9008601.0,0.161363,0.566472,25,0.0,0.0,0.0,0.0,192
4069,riachao do bacamarte,4541.0,PB,1453657.0,0.553,5.0,9008601.0,0.161363,0.556338,25,0.0,0.0,0.0,0.0,217
4068,sao bentinho,4566.0,PB,1453657.0,0.606,5.0,9008601.0,0.161363,0.879177,40,0.0,0.0,0.0,0.0,257
